In [1]:

import cv2
from torch.autograd import Variable
from torchvision.utils import save_image

from loss.LCH import *
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *

In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
def split(img):
    output = []
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [4]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_dtype(torch.float32)

In [5]:
# Initialize generator 
generator = Generator().cuda()
generator.load_state_dict(torch.load("./saved_models/G/generator_795.pth", weights_only=True))

<All keys matched successfully>

In [6]:
generator.eval()

Generator(
  (linear_encoding): Linear(in_features=384, out_features=512, bias=True)
  (position_encoding): LearnedPositionalEncoding()
  (pe_dropout): Dropout(p=0.0, inplace=False)
  (transformer): TransformerModel(
    (net): IntermediateSequential(
      (0): Residual(
        (fn): PreNormDrop(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (fn): SelfAttention(
            (qkv): Linear(in_features=512, out_features=1536, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=512, out_features=512, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (1): Residual(
        (fn): PreNorm(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=512, out_features=256, bias=True)
           

In [7]:
path = './test/input/'  #要改
path_list = os.listdir(path)
path_list.sort(key=lambda x: int(x.split('.')[0]))
i = 1
for item in path_list:
    impath = path + item
    imgx = cv2.imread(path + item)
    imgx = cv2.resize(imgx, (256, 256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx = torch.from_numpy(imgx)
    imgx = imgx.permute(2, 0, 1).unsqueeze(0)
    imgx = imgx / 255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx).cuda()
    #print(imgx.shape)
    output = generator(imgx)
    out = output[3].data
    save_image(out, "./test/output/" + item, nrow=5, normalize=True)
    i = i + 1


In [8]:
def compute_psnr(img1, img2):
    mse = np.mean((img1 / 255. - img2 / 255.) ** 2)
    if mse < 1.0e-10:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


def compute_mse(img1, img2):
    mse = np.mean((img1 / 255. - img2 / 255.) ** 2)
    return mse

In [10]:
path1 = './test/GT/'  #要改
path2 = './test/output/'  #要改
path_list = os.listdir(path1)
path_list.sort(key=lambda x: int(x.split('.')[0]))
PSNR = []

for item in path_list:
    impath1 = path1 + item
    impath2 = path2 + item
    imgx = cv2.imread(impath1)
    imgx = cv2.resize(imgx, (256, 256))
    imgy = cv2.imread(impath2)
    imgy = cv2.resize(imgy, (256, 256))
    #print(imgx.shape)
    psnr1 = compute_psnr(imgx[:, :, 0], imgy[:, :, 0])
    psnr2 = compute_psnr(imgx[:, :, 1], imgy[:, :, 1])
    psnr3 = compute_psnr(imgx[:, :, 2], imgy[:, :, 2])

    psnr = (psnr1 + psnr2 + psnr3) / 3.0

    PSNR.append(psnr)



error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [22]:
PSNR = np.array(PSNR)
print(PSNR.mean())

24.747537285146496
